# Mip Procure Formulation
The problem we formulate in this document has the following characteristics:
- There are multiple SKUs being ordered from a supplier and shipped to a warehouse;
- Inventory of each SKU can be held from one period to the next (there may be inventory at the start of the planning horizon, also known as opening inventory);
- SKUs can be stored both at supplier location or at the warehouse;
- Per-unit inventory cost is charged for each type of SKU;
- There is a minimum quantity (buffer against uncertainty) to be kept in the inventory of each SKU at the warehouse in every period that has demand;
- There is a total inventory capacity at both the warehouse and the supplier;
- Demand for each SKU is known and must be met every period from the warehouse;
- Purchase cost of each SKU is per unit;
- There is a minimum order quantity (MOQ), i.e., if a given SKU is purchased in a given period, then at least, let's say, 100 units must be purchased. Similarly, there's a maximum order quantity;
- There is a minimum shipment quantity for each SKU, i.e., if a given SKU is shipped from the supplier to the warehouse in a given period, then at least, let's say, 100 units of this SKU must be shipped;
- There is an expedition capacity from the supplier, over the number of bags to be shipped per period of all SKUs.
- There is a receiving capacity at the warehouse, set over the number of different **types** of SKUs received (not on their transferred quantities themselves!).

### Additional Complexity: Flexible Receiving Capacity

- Pet Gourmet can now receive up to 30 types of items per week. However, if the number of different items received in a week exceeds 4, an additional cost of R$10,000 is incurred for outsourcing the quality analysis service for the received packaging.

### Input Data Model


#### Modeling Assumptions
1. Inventory levels (both at supplier and warehouse) are assessed at the **end** of time periods.
2. Shipments and purchases are made **throughout** a time period.
3. Deliveries from supplier to warehouse take less than 1 period of time, so we can assume that whatever is purchased in a Period $t$ will get to the warehouse in the same period (as there were no lead time).

#### Indices
- $I$: Collection of items.
- $T$: Collection of time periods.

#### Parameters
- $t_0$: The first time period, that is, $t_0=\min T$.
- $ois_i$: Opening inventory (units) of SKU $i$ at the supplier.
- $oi_i$: Opening inventory (units) of SKU $i$ at the warehouse.
- $cis_i$: Cost (dollar/unit) to keep SKU $i$ in inventory from one period to the next at the supplier.
- $ci_i$: Cost (dollar/unit) to keep SKU $i$ in inventory from one period to the next at the warehouse.
- $il_{it}$: Minimum quantity of SKU $i$ that should be in inventory in Period $t$ at the warehouse.
- $iu_t$: Inventory upper bound (capacity), i.e., maximum total quantity of SKUs that can be kept in inventory in Period $t$ at the warehouse.
- $ius_t$: Inventory upper bound (capacity), i.e., maximum total quantity of SKUs that can be kept in inventory in Period $t$ at the supplier.
- $d_{it}$: Demand (units) of SKU $i$ in Period $t$.
- $cp_i$: Cost (dollar/unit) to purchase SKU $i$.
- $moq_i$: Minimum order quantity of SKU $i$.
- $maxoq_i$: Maximum order quantity of SKU $i$.

- $mtq_i$: Minimum transfer quantity of SKU $i$ from the supplier to the warehouse.
- $tu$: Maximum time that an item can be held in inventory at the supplier (inventory aging).
- $ec$: Expedition capacity at the supplier per period, i.e., a cap on the total amount of **units** that can be shipped from supplier to warehouse across all SKUs.
- $rc_max$: Maximum receiving capacity at the warehouse per period, i.e., a cap over the number of different **types** of SKUs received by the warehouse.
- $rc_threshold$: The threshold of different SKUs (not their quantities!) received by the warehouse before incurring extra cost.
- $extra_cost$: Additional cost incurred if the number of different SKUs received in a period exceeds $rc_threshold$.

### Decision Variables
- $x_{it}$: Quantity (units) of SKU $i$ to be purchased in Period $t$.
- $y_{it}$: Quantity (units) of SKU $i$ kept in inventory from Period $t$ to $t+1$ in the Warehouse.
- $ys_{it}$: Quantity (units) of SKU $i$ kept in inventory from Period $t$ to $t+1$ in the Supplier.
- $z_{it}$: Equals $1$ if SKU $i$ is purchased in Period $t$; $0$ otherwise.
- $w_{it}$: Quantity (units) of SKU $i$ transferred from the supplier to the warehouse in Period $t$.
- $zs_{it}$: Equals $1$ if any quantity (units) of SKU $i$ is transferred from the supplier to the warehouse in Period $t$; $0$ otherwise.
- $q_t$: Equals $1$ if the number of different SKUs received in a period $t$ exceeds $rc_threshold$; $0$ otherwise.

* C0) Initial inventories:  
    a) Opening Inventory at the Supplier:
    $$ ys_{i,t_0-1} = ois_i, \quad \forall i $$
    
    b) Opening Inventory at the Warehouse:
    $$ y_{i,t_0-1} = oi_i, \quad \forall i $$

* C1) Flow balance at the supplier:
$$ys_{i,t-1} + x_{it} = w_{it} + ys_{it}, \quad \forall i, t.$$
The quantity in the supplier inventory from the previous period plus the order of the current period equals the transfer to the warehouse (that departs at the current period) plus the quantity left in inventory for the next period.
    
* C2) Flow balance at the warehouse:
$$y_{i,t-1} + w_{it} = d_{it} + y_{it}, \quad \forall i, t.$$
The quantity in inventory from the previous period plus the transfer of the current period equals the demand plus the quantity left in inventory for the next period.  
Note that this constraint also ensures that demand for SKU $i$ is met in every period. 

* C3) Minimum and maximum order quantities:
$$ moq_i \cdot z_{it} \leq x_{it} \leq maxoq_i \cdot z_{it}, \quad \forall i, t$$
The quantity purchased of SKU $i$ in Period $t$ (i.e., $x_{it}$) is $0$ if it's not bought at all (i.e., if $z_{it}=0$), otherwise it must fall into the minimum and maximum order quantities (i.e., $moq_i \leq x_{it} \leq maxoq_i$).

* C4) Minimum inventory quantity at the warehouse:
$$il_{it} \leq y_{it}, \quad \forall i, t.$$
    
* C5) Inventory capacity:
$$\sum_i y_{it} \leq iu_t, \quad \forall t.$$
$$\sum_i ys_{it} \leq ius_t, \quad \forall t.$$
    
* C6) Minimum transfer size:
$$mtq_i \cdot zs_{it} \leq w_{it} \leq ec \cdot zs_{it}, \quad \forall i, t.$$
If there is a transfer of SKU $i$ in Period $t$ from the supplier to the warehouse, then at least $mtq_i$ units must be transferred, and at most $ec$, since this is the expedition capacity of the supplier (over all SKUs).

* C7) Expedition capacity:
$$ \sum_i w_{it} \leq ec, \quad \forall t. $$
The total number of units transferred from the supplier to the warehouse (over all SKUs) in each period cannot exceed the supplier's expedition capacity.

* C8) Receiving capacity:
$$ \sum_i zs_{it} \leq rc, \quad \forall t. $$
The total number of different SKUs (not their quantities!) transferred from the supplier to the warehouse in each period cannot exceed the warehouse's receiving capacity.

* C9) Maximum inventory aging:
$$ys_{it} \leq \sum_{t+1 \leq t' \leq t+tu}w_{it'}, \quad \forall i, \; t_0-1 \leq t\leq \max T - tu.$$


* C0) Initial inventories:  
    a) Opening Inventory at the Supplier:
    $$ ys_{i,t_0-1} = ois_i, \quad \forall i $$
    
    b) Opening Inventory at the Warehouse:
    $$ y_{i,t_0-1} = oi_i, \quad \forall i $$

* C1) Flow balance at the supplier:
$$ys_{i,t-1} + x_{it} = w_{it} + ys_{it}, \quad \forall i, t.$$
The quantity in the supplier inventory from the previous period plus the order of the current period equals the transfer to the warehouse (that departs at the current period) plus the quantity left in inventory for the next period.
    
* C2) Flow balance at the warehouse:
$$y_{i,t-1} + w_{it} = d_{it} + y_{it}, \quad \forall i, t.$$
The quantity in inventory from the previous period plus the transfer of the current period equals the demand plus the quantity left in inventory for the next period.  
Note that this constraint also ensures that demand for SKU $i$ is met in every period. 

* C3) Minimum and maximum order quantities:
$$ moq_i \cdot z_{it} \leq x_{it} \leq maxoq_i \cdot z_{it}, \quad \forall i, t$$
The quantity purchased of SKU $i$ in Period $t$ (i.e., $x_{it}$) is $0$ if it's not bought at all (i.e., if $z_{it}=0$), otherwise it must fall into the minimum and maximum order quantities (i.e., $moq_i \leq x_{it} \leq maxoq_i$).

* C4) Minimum inventory quantity at the warehouse:
$$il_{it} \leq y_{it}, \quad \forall i, t.$$
    
* C5) Inventory capacity:
$$\sum_i y_{it} \leq iu_t, \quad \forall t.$$
$$\sum_i ys_{it} \leq ius_t, \quad \forall t.$$
    
* C6) Minimum transfer size:
$$mtq_i \cdot zs_{it} \leq w_{it} \leq ec \cdot zs_{it}, \quad \forall i, t.$$
If there is a transfer of SKU $i$ in Period $t$ from the supplier to the warehouse, then at least $mtq_i$ units must be transferred, and at most $ec$, since this is the expedition capacity of the supplier (over all SKUs).

* C7) Expedition capacity:
$$ \sum_i w_{it} \leq ec, \quad \forall t. $$
The total number of units transferred from the supplier to the warehouse (over all SKUs) in each period cannot exceed the supplier's expedition capacity.

* C8) Receiving capacity:
$$ \sum_i zs_{it} \leq rc, \quad \forall t. $$
The total number of different SKUs (not their quantities!) transferred from the supplier to the warehouse in each period cannot exceed the warehouse's receiving capacity.

* C9) Maximum inventory aging:
$$ys_{it} \leq \sum_{t+1 \leq t' \leq t+tu}w_{it'}, \quad \forall i, \; t_0-1 \leq t\leq \max T - tu.$$


### Objective

The goal is to minimize the total purchase and inventory cost:

- $\text{inventory}_{\text{cost}_{\text{s}}} = \sum_{it} cis_i \cdot ys_{it}$,  
- $\text{inventory}_{\text{cost}} = \sum_{it} ci_i \cdot y_{it}$,  
- $\text{purchase}_{\text{cost}} = \sum_{it} cp_i \cdot x_{it}$.

$$\min{ \text{inventory}_{\text{cost}_{\text{s}}} + \text{inventory}_{\text{cost}} + \text{purchase}_{\text{cost}} }.$$

### Additional Constraints for Flexible Receiving Capacity

* C10) Flexible receiving capacity:
Pet Gourmet can now receive up to 30 different items per week. However, if the number of different items received in a week exceeds 4, an additional cost of R$10,000 must be paid for outsourcing the quality analysis of the received packaging.

To implement this in the model, we introduce a new decision variable and an additional constraint:

Let $qc_t$ be a binary variable that equals 1 if more than 4 different items are received in Period $t$, and 0 otherwise.

Then, the constraint can be written as:
$$ qc_t \geq \left( \sum_i zs_{it} - 4 \right)/26, \quad \forall t$$
This constraint ensures that if the number of different SKUs received exceeds 4, the variable $qc_t$ will be set to 1.

The additional cost to the objective function will be:
$$\text{outsourcing}_{\text{cost}} = \sum_t 10,000 \cdot qc_t.$$
The modified objective function becomes:
$$\min{ \text{inventory}_{\text{cost}_{\text{s}}} + \text{inventory}_{\text{cost}} + \text{purchase}_{\text{cost}} + \text{outsourcing}_{\text{cost}} }.$$